# Project: Furuta Pendulum

This notebook provides a starting point for the project, introducing the simulation environment. 

In [ ]:
# Load the autoreload extension to automatically reload modules when they are modified
%load_ext autoreload

# Set autoreload to reload all modules before executing code
%autoreload 2

### Setting Up Dependencies

The simulation and reinforcement learning algorithms used in this project require a few libraries. To install all the necessary dependencies, follow these steps:

1. Open a terminal.
2. Navigate to this directory.
3. Run the following commands (assuming you have already set up the `conda` environment from the exercise sessions):

```
conda activate py13roboticscourse
python -m pip install -r requirements.txt
```

Installing pytorch for Windows may require additional steps. If you encounter any issues, please refer to the official pytorch installation guide. Alternatively, you can work in a windows subsystem for Linux (WSL) environment, which is recommended for this project.

### Load the pendulum configuration

In [ ]:
import yaml 
import os

with open('pendulum_description/simulation_pendulum.yaml', 'r') as file:
        config = yaml.safe_load(file)

parameters_model = config["parameters_model"]
urdf_path = os.path.join("pendulum_description", config["urdf_filename"])
forward_dynamics_casadi_path = os.path.join("pendulum_description", config["forward_dynamics_casadi_filename"])

### Create the environment

In [ ]:
import gymnasium as gym
from furuta_torque_env import FurutaPendulumTorqueEnv

gym.register(
    id="FurutaPendulumTorque-v0",
    entry_point=FurutaPendulumTorqueEnv,
)

# Create an instance of the FurutaPendulumTorqueEnv environment
env = gym.make("FurutaPendulumTorque-v0",
               urdf_model_path=urdf_path, forward_dynamics_casadi_path=forward_dynamics_casadi_path, parameters_model=parameters_model, render=True, swingup=True)

# Click on the url that appears in the output to visualize the environment in a browser

### Execute some control actions on the environment

Random actions are sampled and executed on the pendulum. In the browser visualization, you should see the pendulum moving, until the episode terminates.

In [ ]:
# Reset the environment to its initial state
observation, info = env.reset()

terminated = False
truncated = False
print(f"Initial Observation: {observation}")
# Apply a sequence of actions
while not terminated and not truncated:
    action = env.action_space.sample()  # Sample a random action
    observation, reward, terminated, truncated, info = env.step(action)
    print(f"Observation: {observation}, Reward: {reward}, Terminated: {terminated}, Truncated: {truncated}")

### Load the trained model and test it
We provide a model of an agent that was already been trained to perform upward stabilization of the pendulum. 


This model was trained for 1 million timesteps using a slightly modified version of the [CleanRL implementation of PPO for continuous actions](https://docs.cleanrl.dev/rl-algorithms/ppo/#ppo_continuous_actionpy). You can find the script used for training (`ppo_continuous_action.py`) in this same directory.
In the following cells, we demonstrate how to load the trained model and visualize its behavior in the environment.

In [ ]:
import torch
from ppo_continuous_action import Agent, make_env

saved_model_path = 'furuta_pendulum_tensorboard/furutaTorque__ppo_continuous_action__42__1744890705/ppo_continuous_action.cleanrl_model' # no swingup

# Set to False if task includes swingup
swingup = False

envs = gym.vector.SyncVectorEnv(
        [make_env(urdf_path=urdf_path, 
                  parameters_model=parameters_model, 
                  forward_dynamics_casadi_path=forward_dynamics_casadi_path,
                  render=True, swingup=swingup) for _ in range(1)]
    )

model = Agent(envs=envs)

model.load_state_dict(torch.load(saved_model_path, map_location="cpu"))
model.eval()

In [ ]:
# Test the trained model
observation, info = envs.reset()
terminated = False
truncated = False
print(f"Initial Observation: {observation}")
# Apply a sequence of actions
while not terminated and not truncated:
    action, _, _, _ = model.get_action_and_value(torch.Tensor(observation), deterministic=True)  # Use the trained model to predict the action
    action = action.cpu().detach().numpy()
    observation, reward, terminated, truncated, info = envs.step(action)
    print(f"Observation: {observation}, Action: {action}, Reward: {reward}, Terminated: {terminated}, Truncated: {truncated}")
# Close the environment
envs.close()


### Tensorboard

Below you can see the charts of the training evolution, visualized through Tensorboard. These charts are generated automatically when training a model using the `ppo_continuos_action.py` script.


In [ ]:
%load_ext tensorboard

%tensorboard --logdir furuta_pendulum_tensorboard